# AWS AI Week : OpenCALM SageMaker Fine-tuning

SageMaker上でLoRAにより [OpenCALM](https://huggingface.co/cyberagent/open-calm-7b) を Fine-tuning し、デプロイするサンプルコードです。
- このデモでは、Instruction tuning 形式で Fine-tuning を行います。
- [JAQKET](https://www.nlp.ecei.tohoku.ac.jp/projects/jaqket/) データセットを使用しています。

- 以下のようなユースケースを想定しています。
  - 日本語対応の大規模言語モデルを使いたい。
  - 効率よくInstruction tuning を行い、QA の仕方を学習させる。

SageMaker Studio のノートブックカーネルは、 Data Science 2.0、Python 3 で動作確認済みです。

---

まず、必要なパッケージをインストールしてアップグレードします。以下のセルを初回に実行したら、カーネルを再起動します。

---

In [ ]:
!pip install -U "sagemaker>=2.143.0"

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## 学習用データセットの準備
---
Fine-Tuning 用の日本語データをダウンロードします。

---

In [ ]:
!wget -P data https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_02_train.jsonl

In [ ]:
!head -n 2 data/aio_02_train.jsonl

In [ ]:
# Convet .jsonl to .json
import pandas as pd

# 各行を１レコードとして読み込み、列名を question -> instruction, answers -> output にリネームする。
df = pd.read_json("data/aio_02_train.jsonl", orient="records", lines=True)
df = df.rename(columns={"question": "instruction", "answers": "output"})
df = df[["instruction", "output"]]
df["output"] = df["output"].apply(lambda x: f"{x[0]}」")
df["input"] = ""
print(df.shape)
df.to_json(
    "data/aio_02_train_formatted.jsonl", orient="records", force_ascii=False, lines=True
)

In [ ]:
df.head(2)

---

データセットを先頭の1,000件のみ取り出します。

- 以下を実行したら、直接作成した json ファイルの内容も確認してみてください。
- 元データ (aio_02_train_formatted.jsonl)、および、作成済みデータ (aio_02_train_formatted_1000.jsonl) は、「data」フォルダに格納されます。

---

In [ ]:
# 1000 行のデータを取り出す。
df_1000 = df.iloc[:1000]
df_1000.to_json("data/aio_02_train_formatted_1000.jsonl", orient="records", force_ascii=False, lines=True)

In [ ]:
print(df_1000.shape)

---

データセットをS3へアップロードします。

---

In [ ]:
# 学習させるデータを指定してS3へアップロード
train_jsonpath = "./data/aio_02_train_formatted_1000.jsonl"

input_train = sess.upload_data(
    path=train_jsonpath, key_prefix="OpenCALM"
)
input_train

## Fine-tuning を実行
---
作成したデータセットを使用して、Fine-tuning を行います。

今回は、[HuggingFace estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-estimator) (推定器) を使用して、学習を行います。

- entry_point : 学習のエントリポイントとして実行する Python ソースファイルへのパス(source＿dir)
- instance_type : 学習で使用するインスタンスタイプを指定

主なハイパーパラメータを説明します。
- base_model : Fine-tuning に使用するモデルを指定
- data_path : Fine-tuning に使用するデータを指定
- num_epochs : 1 epochは、データセット全体が1回だけ渡される(小さなバッチに分割して送信される)
- LoRA (Low Rank Adapter)
  凍結した事前学習済みモデルに低ランク行列を追加し、低ランク行列に対してパラメータ更新する手法。
  - lora_target_modules : チューニング用のパラメーターを用意する
  - lora_r : LoRA の行列ランクであり、小さいほど学習によって更新されるパラメータは少なくなる。
- prompt_template_name : プロンプトのテンプレートを指定(scripts/code/templates/)

---

In [ ]:
jsonfile = train_jsonpath.split("/")[-1]
data_path = "/opt/ml/input/data/train/" + jsonfile

base_model_name = "open-calm-7b"
base_model = "cyberagent/open-calm-7b"

In [ ]:
hyperparameters={
    'base_model': base_model, # OpenCALM 7B モデルを使用
    # 'load_in_8bit': True,
    'load_in_4bit': True,
    'pad_token_id': 1,
    'data_path': data_path, # 要素を調整したデータセットを指定
    'num_epochs': 1, # default 3
    'cutoff_len': 512,
    'group_by_length': False,
    'output_dir': '/opt/ml/model',
    # 'resume_from_checkpoint': '/opt/ml/checkpoints',
    'lora_target_modules': '[query_key_value]',
    'lora_r': 16,
    'batch_size': 32,
    'micro_batch_size': 4,
    'prompt_template_name': 'simple_qa_ja',
}

In [ ]:
huggingface_estimator = HuggingFace(
    base_job_name = base_model_name,
    role=role,
    entry_point='finetune_ai-week.py',
    source_dir='./scripts/code',
    # instance_type='ml.g5.2xlarge',
    instance_type='ml.p3.2xlarge', # 試験的な実施のため、小さい GPU インスタンスに変更
    instance_count=1,
    volume_size=200,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    use_spot_instances=False,
    # use_spot_instances=True, # スポットインスタンスを利用する場合は True にし、max_wait を有効にする。
    # max_wait=86400,
    hyperparameters=hyperparameters,
    metric_definitions=[{'Name': 'eval_loss', 'Regex': "'eval_loss': (\d\.\d+)"},
                        {'Name': 'train_loss', 'Regex': "'loss': (\d\.\d+)"}],
    # checkpoint_s3_uri=f"s3://{bucket}/{base_job_name}/checkpoint/",
)
huggingface_estimator.fit({'train': input_train})

## トレーニングされたモデルを取得
---

In [ ]:
import boto3
import sagemaker

# 最後に実行されたトレーニングジョブで出力されたアーティファクトを取得して S3 に保存
def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data = huggingface_estimator.model_data
except:
    # カーネルがリスタートした時にアーティファクトの　URL を取得
    model_data = get_latest_training_job_artifact('OpenCALM')
    
!aws s3 cp {model_data} opencalm.tar.gz

In [ ]:
!rm -rf scripts/model && mkdir scripts/model
!tar -xvf opencalm.tar.gz -C scripts/model --no-same-owner --wildcards adapter_*
!ls -l scripts/model

## モデルをパッケージして S3 へアップロード
---

In [ ]:
%cd scripts
!tar -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"OpenCALM")
model_path

## パッケージ化された Fine-tuning 済みのモデルをデプロイする
---
リアルタイムエンドポイントを作成し、エンドポイント経由で推論できるようにデプロイします。

[リアルタイム推論](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/realtime-endpoints.html)は、リアルタイム、インタラクティブ、ミリ秒オーダーの低レイテンシーが要求されるワークロードに最適です。ペイロードサイズは最大 6 MB です。

デプロイオプションとして、[非同期推論 (async inference)](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/async-inference.html) を行うこともできます。

非同期推論もエンドポイント経由でモデルへアクセスしますが、リクエストをキューに配置して非同期で処理します。ペイロードサイズがリアルタイム推論よりも大きい(最大 1 GB )、処理時間が長い(最大 1 時間)、ほぼリアルタイムの要求に最適です。処理するリクエストがない場合は、オートスケーリングで最少インスタンス数をゼロにすることができるため、コストを節約する事ができます。

---

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer


huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=base_model_name,
    env={
        "model_params": json.dumps({
            "base_model": base_model, # モデルを指定
            "lora_weights": "model", # モデルパッケージを取得するためのパス
            "peft": True,
            "load_4bit": True,
            "prompt_template": "simple_qa_ja",
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# SageMaker 推論として、モデルをデプロイする
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=base_model_name,
    serializer=JSONSerializer(),
    # 非同期推論の場合は、以下オプションを使用する
    # async_inference_config=AsyncInferenceConfig()
)

## 推論を実行
---
実際に推論を行なってみます。

今回は、クイズ形式のデータセットで Instruction tuning を行ったため、単語で答えられる質問を行います。
instruction を変更し、様々な入力を試してください。
- max_new_tokens: モデルは、出力長 (入力コンテキストの長さを除く) が max_new_tokens に達するまでテキストを生成する。指定する場合は正の整数でなければならない。
- temperature: 出力のランダム性を制御する。temperature が高いと確率の低い単語での出力になり(より創造的)、temperature が低いと確率の高いワードで出力される。
- top_p: テキスト生成の各ステップで、累積確率が top_p をもつ最小の単語セットからサンプリングする。(0.1の場合は、確率が上位10%を考慮。) 値は、0〜1 の浮動小数点数。
- return_full_text: True の場合、入力テキストが出力テキストの一部として出力される。

---

In [ ]:
# With SageMaker SDK

from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client = Predictor(
    endpoint_name=base_model_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

- インプット例 ：

  - Q：代表的な解熱鎮痛剤のひとつで非ステロイド性抗炎症薬の代名詞とも言うべき医薬品で、ドイツのバイエルが命名し、1899年に商標登録された薬品名は？
    - A：アスピリン  
  - Q：「充分な説明に基づく合意」を意味する、医療行為を行うにあたって事前に診療の内容を患者に説明し、了承を得ることを英語で何というでしょう?
    - A： インフォームドコンセント
  - Q：歯磨き粉や日焼け止めクリームなど、医薬品と化粧品の中間に位置するものを何というでしょう?
    - A：医薬部外品
  - Q：新しく開発された薬の安全性や効き目などを調べるため、患者の治療を兼ねて行う試験のことを何試験と言うでしょう?
    - A：臨床試験

また、/data に入っている、「aio_02_train_formatted.jsonl」を開いてみてください。
今回、先頭の1000までしか学習させていませんので、その後ろのデータも質問として利用できます。

In [ ]:
#　答え：アスピリン
data = {
    "instruction": "代表的な解熱鎮痛剤のひとつで非ステロイド性抗炎症薬の代名詞とも言うべき医薬品で、ドイツのバイエルが命名し、1899年に商標登録された薬品名は？",
    "max_new_tokens": 120,
    "temperature": 0.3,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "repetition_penalty": 1.05,
    # "top_p": 0.75,
    # "top_k": 40,
    # "no_repeat_ngram_size": 2,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)

print("答えは「" + response)

In [ ]:
# 答え：インフォームドコンセント

data = {
    "instruction": "「充分な説明に基づく合意」を意味する、医療行為を行うにあたって事前に診療の内容を患者に説明し、了承を得ることを英語で何というでしょう?",
    "max_new_tokens": 120,
    "temperature": 0.3,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "repetition_penalty": 1.05,
    # "top_p": 0.75,
    # "top_k": 40,
    # "no_repeat_ngram_size": 2,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)
print("答えは「" + response)

## ベンチマーク： Speed

In [ ]:
%timeit response = predictor_client.predict(data=data)

## エンドポイントを削除 (クリーニング)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()